<a href="https://colab.research.google.com/github/dkchebet7/job_compatibility_system/blob/main/Job_Compatibility_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# 
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Our datasets are stored in an excel file with different sheets
# First, we'll read the dataset and then create dataframes from the differents sheets

df = pd.ExcelFile('Capstone Project Data Set.xlsx')
candidates_df = pd.read_excel(df, 'Candidates')
job_df = pd.read_excel(df, 'Job Details')

In [ ]:
# Creating a function that will clean the column names

def cleaned_columns(dataframe):
    dataframe.columns = dataframe.columns.str.lower().str.replace(' ', '_')

In [ ]:
# Applying the function to the job details dataframe
cleaned_columns(job_df)
job_df.columns

Index(['job_id', 'job_title', 'keywords', 'minimum_experience_in_years',
       'maximum_experience_in_years', 'minimum_annual_salary',
       'maximum_annual_salary', 'currency', 'must_haves', 'company_name',
       'founded_year', 'employee_count', 'job_location', 'industry',
       'function', 'required_education_(ug)', 'required_education_(pg)',
       'notice_period_(days)'],
      dtype='object')

In [ ]:
# Applying the function to the candidates dataframe
cleaned_columns(candidates_df)
candidates_df.columns

Index(['candidate_id', 'location', 'current_annual_salary',
       'current_salary_currency_', 'current_salary_type',
       'expected_annual_salary', 'expected_salary_currency',
       'total_experience_in_years', 'current_industry', 'current_fuction',
       'relocation_cities_', 'notice_period_in_days', 'current_job_title',
       'data_source'],
      dtype='object')

In [ ]:
# Checking for null values in the dataframes

candidates_df.isnull().sum()

candidate_id                    0
location                       82
current_annual_salary        1687
current_salary_currency_     3405
current_salary_type             1
expected_annual_salary       1115
expected_salary_currency      577
total_experience_in_years     525
current_industry                0
current_fuction              1243
relocation_cities_           6804
notice_period_in_days        2852
current_job_title             442
data_source                     0
dtype: int64

In [ ]:
job_df.isnull().sum()

job_id                         0
job_title                      0
keywords                       1
minimum_experience_in_years    0
maximum_experience_in_years    0
minimum_annual_salary          0
maximum_annual_salary          0
currency                       0
must_haves                     1
company_name                   0
founded_year                   0
employee_count                 0
job_location                   0
industry                       0
function                       0
required_education_(ug)        0
required_education_(pg)        3
notice_period_(days)           0
dtype: int64

#### Processing the Job details Dataframe

In [ ]:
# Previewing the DataFrame
job_df

,job_id,job_title,keywords,minimum_experience_in_years,maximum_experience_in_years,minimum_annual_salary,maximum_annual_salary,currency,must_haves,company_name,founded_year,employee_count,job_location,industry,function,required_education_(ug),required_education_(pg),notice_period_(days)
0,194,Full stack engineer,"java,Angular,Jquery,j2ee,microservices,micro s...",2,5,6,15,INR,NaN,Company X,2011,362,Hyderabad,IT | Computer Software,IT Software,B.Tech/B.E.,NaN,60
1,481,Senior Java Engineer/Tech Lead,"J2EE,Java,Hibernate,Webervices,REST Services,S...",7,10,10,15,INR,Should have proficient Java server side progra...,Company X,2011,362,Hyderabad,IT | Computer Software,IT Software,Other UG,Not Required,30
2,839,Front End Developer,NaN,2,5,4,8,INR,The candidate must have at least 2 years of so...,Company X,2011,362,Bengaluru/Bangalore,IT | Information Technology and Services,IT Software,Undergraduate (engineering),NaN,30
3,723,Application Developer,"c#, asp.net, sql, java, html5",3,8,4,9,INR,The Application Developer is a role within the...,Company X,2011,362,Bengaluru,IT | Information Technology and Services,IT Software,B.Tech/B.E.,NaN,30


In [ ]:
# Filling the null values in the 'required_education' column using the 
# forward fill and back fill method

job_df['required_education_(pg)'] = job_df['required_education_(pg)'].ffill().bfill()


In [ ]:
# Filling the null value in the 'keywords' column 
# We will use common skills associated with Front End Developers sourced from 
# the internet
# We assign these skills to a variable x
x = ('html,javascript,jquery,javascrit,css,bootstrap,version control,git,gui')
job_df['keywords'].fillna(x, inplace = True)

In [ ]:
# Creating a copy of this job details dataframe and call it job
job = job_df.copy()
job

,job_id,job_title,keywords,minimum_experience_in_years,maximum_experience_in_years,minimum_annual_salary,maximum_annual_salary,currency,must_haves,company_name,founded_year,employee_count,job_location,industry,function,required_education_(ug),required_education_(pg),notice_period_(days)
0,194,Full stack engineer,"java,Angular,Jquery,j2ee,microservices,micro s...",2,5,6,15,INR,NaN,Company X,2011,362,Hyderabad,IT | Computer Software,IT Software,B.Tech/B.E.,Not Required,60
1,481,Senior Java Engineer/Tech Lead,"J2EE,Java,Hibernate,Webervices,REST Services,S...",7,10,10,15,INR,Should have proficient Java server side progra...,Company X,2011,362,Hyderabad,IT | Computer Software,IT Software,Other UG,Not Required,30
2,839,Front End Developer,"html,javascript,jquery,javascrit,css,bootstrap...",2,5,4,8,INR,The candidate must have at least 2 years of so...,Company X,2011,362,Bengaluru/Bangalore,IT | Information Technology and Services,IT Software,Undergraduate (engineering),Not Required,30
3,723,Application Developer,"c#, asp.net, sql, java, html5",3,8,4,9,INR,The Application Developer is a role within the...,Company X,2011,362,Bengaluru,IT | Information Technology and Services,IT Software,B.Tech/B.E.,Not Required,30


In [ ]:
# Now we can merge the text data columns to make their processing simpler
job['job_details'] = job[['keywords','currency','must_haves','job_location',
                           'industry','function','required_education_(ug)',]].values.tolist()

In [ ]:
# We then drop these columns
job = job.drop(['keywords','currency','must_haves','job_location',
                'industry','function','required_education_(ug)'], axis=1)

In [ ]:
# Creating a separate dataframe with the numerical information
job_numerical_df = job.drop(['job_title', 'company_name', 'required_education_(pg)',
                                     'job_details'], axis = 1)
job_numerical_df

,job_id,minimum_experience_in_years,maximum_experience_in_years,minimum_annual_salary,maximum_annual_salary,founded_year,employee_count,notice_period_(days)
0,194,2,5,6,15,2011,362,60
1,481,7,10,10,15,2011,362,30
2,839,2,5,4,8,2011,362,30
3,723,3,8,4,9,2011,362,30


In [ ]:
# Now lets drop the columns that are unnecessary for our analysis
job = job.drop(['minimum_experience_in_years','maximum_experience_in_years',
                'minimum_annual_salary','maximum_annual_salary',
                'company_name','founded_year','employee_count',
                'required_education_(pg)','notice_period_(days)'], axis=1)
job

,job_id,job_title,job_details
0,194,Full stack engineer,"[java,Angular,Jquery,j2ee,microservices,micro ..."
1,481,Senior Java Engineer/Tech Lead,"[J2EE,Java,Hibernate,Webervices,REST Services,..."
2,839,Front End Developer,"[html,javascript,jquery,javascrit,css,bootstra..."
3,723,Application Developer,"[c#, asp.net, sql, java, html5, INR, The Appli..."


In [ ]:
# Cleaning up the job_details column a little further
job['details'] = (
 pd.DataFrame(job['job_details'].tolist())
   .fillna('')
   .astype(str)
   .agg(','.join, 1)
   .str.replace(',', ' ')
)
job

,job_id,job_title,job_details,details
0,194,Full stack engineer,"[java,Angular,Jquery,j2ee,microservices,micro ...",java Angular Jquery j2ee microservices micro s...
1,481,Senior Java Engineer/Tech Lead,"[J2EE,Java,Hibernate,Webervices,REST Services,...",J2EE Java Hibernate Webervices REST Services S...
2,839,Front End Developer,"[html,javascript,jquery,javascrit,css,bootstra...",html javascript jquery javascrit css bootstrap...
3,723,Application Developer,"[c#, asp.net, sql, java, html5, INR, The Appli...",c# asp.net sql java html5 INR The Applicat...


In [ ]:
job = job.drop(['job_details'], axis=1)
job

,job_id,job_title,details
0,194,Full stack engineer,java Angular Jquery j2ee microservices micro s...
1,481,Senior Java Engineer/Tech Lead,J2EE Java Hibernate Webervices REST Services S...
2,839,Front End Developer,html javascript jquery javascrit css bootstrap...
3,723,Application Developer,c# asp.net sql java html5 INR The Applicat...


#### Processing the candidates dataframe

In [ ]:
# We'll generally take the same steps we carried out for the job_details dataframe
# First Let's preview the top o the dataframe
candidates_df.head()

,candidate_id,location,current_annual_salary,current_salary_currency_,current_salary_type,expected_annual_salary,expected_salary_currency,total_experience_in_years,current_industry,current_fuction,relocation_cities_,notice_period_in_days,current_job_title,data_source
0,A13380,NCR,NaN,NaN,ANNUAL,60.0,INR,9.00,"Banking,Financial Services,Broking",NaN,NaN,NaN,Team Manager Insurance,INDEED.COM
1,A9339,Vadodara,8.0,INR,ANNUAL,8.0,INR,3.00,Energy and Utilities | Telecommunications#Manu...,Business Development#Manufacturing#Project Man...,NaN,30.0,Marketing Executive,INTERNAL DATABASE
2,A10220,Noida,11.0,INR,ANNUAL,11.0,INR,0.70,IT | Computer Software#Manufacturing | Mechani...,IT Software,NaN,90.0,Software Developer,INTERNAL DATABASE
3,A8162,Hyderabad,8.0,INR,ANNUAL,8.0,INR,3.84,"IT | Information Services, IT | Computer Software","Business Development, IT Software",NaN,90.0,Senior systems engineer,LINKEDIN
4,A13729,Hyderabad,9.6,NaN,ANNUAL,11.0,INR,2.50,IT | Computer Software,IT Software#Research,NaN,30.0,Software Developer,LINKEDIN


In [ ]:
# We'll first create a copy of the dataframe
candidates = candidates_df.copy()

In [ ]:
# Chaging 'monthly' to 'annual'
a = candidates[candidates['current_salary_type']=="MONTHLY"]
b = candidates[candidates['current_salary_type']!="MONTHLY"]
a.expected_annual_salary = a.expected_annual_salary *12
dfs = [a, b]
candidates = pd.concat(dfs)
candidates.shape

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


(7006, 14)

In [ ]:
# Assuming present day exchange rate
# converting ksh to inr
a = candidates[candidates['current_salary_currency_']=="KSH"]
b = candidates[candidates['current_salary_currency_']!="KSH"]
a.expected_annual_salary = a.expected_annual_salary *0.7
dfs = [a, b]
candidates = pd.concat(dfs)
candidates.shape

(7006, 14)

In [ ]:
# converting usd to inr
a = candidates[candidates['current_salary_currency_']=="KSH"]
b = candidates[candidates['current_salary_currency_']!="KSH"]
a.expected_annual_salary = a.expected_annual_salary *0.7
dfs = [a, b]
candidates = pd.concat(dfs)
candidates.shape

(7006, 14)

In [ ]:
# Merging the text data columns
candidates['candidates_details'] = candidates[['location','current_industry',
                        'current_fuction','current_job_title']].values.tolist()

In [ ]:
# Dropping the merged columns
candidates = candidates.drop(['location','current_industry',
                        'current_fuction','current_job_title'], axis = 1)

In [ ]:
# Creating a dataframe with the candidates numerical info
candidates_numerical_df = candidates.drop(['current_salary_currency_','current_salary_type', 'expected_salary_currency',
                                          'relocation_cities_', 'data_source','candidates_details'], axis = 1)
candidates_numerical_df.head()

,candidate_id,current_annual_salary,expected_annual_salary,total_experience_in_years,notice_period_in_days
4164,A7700,18000.00,8820.0,4.35,90.0
20,A14341,36.00,504.0,2.40,15.0
470,A14378,0.00,0.0,1.00,0.0
1037,A14282,7.68,136.8,3.60,30.0
1100,A14136,936.00,21600.0,7.00,15.0


In [ ]:
# Now let's drop the columns unnecessary for analysis
candidates = candidates.drop(['current_annual_salary','current_salary_currency_','current_salary_type','expected_annual_salary',
                              'expected_salary_currency','total_experience_in_years','relocation_cities_','notice_period_in_days',
                              'data_source'], axis = 1)
candidates.head()

,candidate_id,candidates_details
4164,A7700,"[bhubaneswar, IT | Computer & Network Security..."
20,A14341,"[Vellore, IT | Information Technology and Serv..."
470,A14378,"[Achalpur, Ads, PR, Events | Events Services#A..."
1037,A14282,"[New Delhi, IT | Computer Software#IT | Inform..."
1100,A14136,"[Kakamega, IT | Computer Software#Entertainmen..."


In [ ]:
# Cleaning up th details column
candidates['details'] = (
 pd.DataFrame(candidates['candidates_details'].tolist())
   .fillna('')
   .astype(str)
   .agg(','.join, 1)
   .str.replace(',', ' ')
)
candidates.head()

,candidate_id,candidates_details,details
4164,A7700,"[bhubaneswar, IT | Computer & Network Security...",Pune IT | Information Technology and Services ...
20,A14341,"[Vellore, IT | Information Technology and Serv...",Gurgaon IT | Information Technology and Servic...
470,A14378,"[Achalpur, Ads, PR, Events | Events Services#A...",Pune Entertainment Media | Newspapers#Financi...
1037,A14282,"[New Delhi, IT | Computer Software#IT | Inform...",Mumbai Consumer | Food & Beverages#Consumer | ...
1100,A14136,"[Kakamega, IT | Computer Software#Entertainmen...",Hyderabad IT | Computer Software IT Software S...


In [ ]:
# the final candidates dataframe
candidates = candidates.drop(['candidates_details'], axis = 1)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Creating a function that cleans both 'details' column in the two dataframes

stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def tokenizer(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def processed_text(details):
  clean_text = []
  clean_text2 = []
  details = re.sub("'", "",details)
  details=re.sub("(\\d|\\W)+"," ",details) 
  # details = details.replace("nbsp", "")
  clean_text = [wn.lemmatize(word, pos="n") for word in word_tokenize(details.lower()) if tokenizer(word)]
  clean_text2 = [word for word in clean_text if tokenizer(word)]
  return " ".join(clean_text2)

In [ ]:
# Applying the function to the candidates dataframe
candidates['details'] = candidates['details'].apply(processed_text)
candidates.head()

,candidate_id,details
4164,A7700,pune information technology service software p...
20,A14341,gurgaon information technology service softwar...
470,A14378,pune entertainment medium newspaper financial ...
1037,A14282,mumbai consumer food beverage consumer wine sp...
1100,A14136,hyderabad computer software software automation


In [ ]:
# Applying the function to the job dataframe
job['details'] = job['details'].apply(processed_text)
job.head()

,job_id,job_title,details
0,194,Full stack engineer,java angular jquery microservices micro servic...
1,481,Senior Java Engineer/Tech Lead,java hibernate webervices rest service spring ...
2,839,Front End Developer,html javascript jquery javascrit bootstrap ver...
3,723,Application Developer,asp net sql java html inr application develope...


#### Content based Recommender with tfidf

In [ ]:
# carrying out feature extraction using tfidf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
# 
job_tfidf = tfidf.fit_transform(job['details'])
job_tfidf

<4x299 sparse matrix of type '<class 'numpy.float64'>'
	with 336 stored elements in Compressed Sparse Row format>

In [ ]:
# Performing feature extraction on the candidates dataframe
candidates_tfidf = tfidf.transform(candidates['details'])
candidates_tfidf

<7006x299 sparse matrix of type '<class 'numpy.float64'>'
	with 29569 stored elements in Compressed Sparse Row format>

In [ ]:
# Performing Cosine Similarity to measure how similar the two dataframes are
# The idea here is that if the cosine is close to 1, the items are similar and 
# they are close to 0, they are not similar. In our case this will show us how 
# much a candidate matches to a particular job

from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(candidates_tfidf, job_tfidf)
# cs = map(lambda x: cosine_similarity(candidates_tfidf, x), job_tfidf)

In [ ]:
# Checking the shape of the final dataframe

cs.shape

(7006, 4)

In [ ]:
cols=['Full stack engineer', 'Senior Java Engineer/Tech Lead', 'Front End Developer', 'Application Developer']

In [ ]:
# Creating a scores dataframe
candidates_index = pd.Series(candidates.candidate_id)
job_index = pd.Series(job.job_title)
# 
scores_df = pd.DataFrame(cs, index = candidates_index, columns = job_index)
scores_df.head()

In [ ]:
# Checking the highest score in the 'full stack engineer' column
scores_df['Full stack engineer'].max()

0.4626483336319382

In [ ]:
# Getting all the information of the above candidate

scores_df[scores_df['Full stack engineer']== 0.4626483336319382]

job_title,Full stack engineer,Senior Java Engineer/Tech Lead,Front End Developer,Application Developer
candidate_id,,,,
A11923,0.462648,0.396634,0.15674,0.060786


In [ ]:
# Let's save this dataframe in our local library
scores_df.to_csv('Job_Compatibility_Scoring_System.csv')

In [ ]:
scores_columns

Index(['Full stack engineer', 'Senior Java Engineer/Tech Lead',
       'Front End Developer', 'Application Developer'],
      dtype='object', name='job_title')

In [ ]:
scores_df = pd.read_csv('Job_Compatibility_Scoring_System.csv')
final_scores.head()

,candidate_id,Full stack engineer,Senior Java Engineer/Tech Lead,Front End Developer,Application Developer
0,A7700,0.091284,0.067389,0.180828,0.217258
1,A14341,0.153216,0.113109,0.395087,0.297133
2,A14378,0.000000,0.000000,0.000000,0.236807
3,A14282,0.052986,0.039116,0.184134,0.295318
4,A14136,0.423694,0.312784,0.171151,0.041517


In [ ]:
for job in scores_columns:
    a = scores_df[job].max()
    print(a)

0.4626483336319382
0.4006775880620435
0.48360476474282516
0.4733804309944653


In [ ]:
def recommend(scores_columns):
    for job in scores_columns:
        a = scores_df[job].max()
        print(job+':', a,':', 'Candidate ID', scores_df[scores_df[job]==a]['candidate_id'].values)

In [ ]:
recommend(['Full stack engineer', 'Senior Java Engineer/Tech Lead', 'Front End Developer', 'Application Developer'])

Full stack engineer: 0.4626483336319382 : Candidate ID ['A11923']
Senior Java Engineer/Tech Lead: 0.4006775880620435 : Candidate ID ['A10693']
Front End Developer: 0.4836047647428252 : Candidate ID ['A12003']
Application Developer: 0.4733804309944653 : Candidate ID ['A11952']
